# Trabájo Practico 2 - Métodos Computacionales

### Camila Cauzzo, Catalina Dolhare

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import time
import random

In [2]:
from derivada_w import fAndG as fAndG_w
from derivada_b2 import fAndG as fAndG_b

In [3]:
def abrirImagenesEscaladas( carpeta, escala=32 ):
    # abre todas las imagenes de la carpeta, y las escala de tal forma que midan (escala x escala)px
    # devuelve las imagenes aplanadas -> vectores de tamano escala^2 con valores entre 0 y 1
    imagenes = []

    for dirpath, dirnames, filenames in os.walk(carpeta):
        for file in filenames:
            img = Image.open( os.path.join(carpeta, file) )
            img = img.resize((escala, escala))
            img.convert('1')
            img = np.asarray(img)
            if len(img.shape)==3:
                img = img[:,:,0].reshape((escala**2 )) / 255
            else:
                img = img.reshape((escala**2 )) / 255
            
            imagenes.append( img )

    return imagenes

In [4]:
def balancear_datos():
    imagenes_train_normal = abrirImagenesEscaladas("chest_xray/train/NORMAL")
    imagenes_train_pneumonia = abrirImagenesEscaladas("chest_xray/train/PNEUMONIA")

    cantidad_imagenes = min(len(imagenes_train_normal), len(imagenes_train_pneumonia))
    imagenes_train_normal = imagenes_train_normal[:cantidad_imagenes]
    imagenes_train_pneumonia = imagenes_train_pneumonia[:cantidad_imagenes]

    return [imagenes_train_normal, imagenes_train_pneumonia]

### Ejercicio 1

Como la derivada de la suma es la suma de las derivadas, sumamos las derivadas de cada imagen

In [5]:
def f(i, w, b):
    return (np.tanh(np.dot(w, i) + b)+1)/2

In [6]:
imagenes = balancear_datos()

In [7]:
print(len(imagenes[0]), len(imagenes[1]))

1349 1349


In [8]:
def L(w, b):
    imagenes_normal = imagenes[0]
    imagenes_pneumonia = imagenes[1]

    sumL = 0

    for i in imagenes_normal:
        L = (f(i, w, b) - 0)**2
        sumL += L
    
    for i in imagenes_pneumonia:
        L = (f(i, w, b) - 1)**2
        sumL += L
        
    return sumL

In [9]:
def desc_grad(w, b, alpha, grad_w, grad_b):
    w = w - alpha * grad_w
    b = b - alpha * grad_b
    return w, b

In [10]:
def entrenamiento(imagenes):
    imagenes_normal = imagenes[0]
    imagenes_pneumonia = imagenes[1]

    w = np.zeros(imagenes[0][0].shape)
    b = 0

    alpha = 0.0001

    for i in range(1000):
        print("Iteración: ", i, "- Mínimo alcanzado hasta el momento: ", L(w, b))
        sum_dw = np.zeros(imagenes[0][0].shape)
        sum_db = 0

        for i in imagenes_normal:
            sum_dw += fAndG_w(b, 0, i, w)[1]
            sum_db += fAndG_b(b, 0, i, w)[1]
        
        for i in imagenes_pneumonia:
            sum_dw += fAndG_w(b, 1, i, w)[1]
            sum_db += fAndG_b(b, 1, i, w)[1]

        if abs(L(w - alpha * sum_dw, b - alpha * sum_db)-L(w, b)) < 0.00001:
            break
        
        w, b = desc_grad(w, b, alpha, sum_dw, sum_db)
        
        print(L(w, b))
    
    return w, b

In [11]:
w_opt, b_opt = entrenamiento(imagenes)
print("El mínimo local ocurre en w = ", w_opt, "b = ", b_opt, " y L vale: ", L(w_opt, b_opt))

Iteración:  0 - Mínimo alcanzado hasta el momento:  674.5
720.390056804488
Iteración:  1 - Mínimo alcanzado hasta el momento:  720.390056804488
1348.999999861933
Iteración:  2 - Mínimo alcanzado hasta el momento:  1348.999999861933
El mínimo local ocurre en w =  [-0.01102996 -0.01530102 -0.02131791 ... -0.02286529 -0.00583192
  0.00294252] b =  -0.05902200858208572  y L vale:  1348.999999861933


In [12]:
def derivada_w(b, imagenes, w):
    imagenes_normal = imagenes[0]
    imagenes_pneumonia = imagenes[1]
    
    sum_dw = np.zeros(imagenes[0][0].shape)
    for i in range(len(imagenes_normal)):
        dw = fAndG_w(b, 0, imagenes_normal[i], w)
        sum_dw += dw[1]
    for i in range(len(imagenes_pneumonia)):
        dw = fAndG_w(b, 1, imagenes_pneumonia[i], w)
        sum_dw += dw[1]
    return sum_dw

def derivada_b(b, imagenes, w):
    imagenes_normal = imagenes[0]
    imagenes_pneumonia = imagenes[1]
    
    sum_db = 0
    for i in range(len(imagenes_normal)):
        db = fAndG_b(b, 0, imagenes_normal[i], w)
        sum_db += db[1]
    for i in range(len(imagenes_pneumonia)):
        db = fAndG_b(b, 1, imagenes_pneumonia[i], w)
        sum_db += db[1]
    return sum_db


### Ejercicio 2

Una vez que tenemos las derivadas parciales, podemos utilizar el metodo de gradiente descendiente

In [13]:
MAX_ITER = 1000 # maximo numero de iteraciones
TOLERANCIA = 0.0000001 # tolerancia para la convergencia

In [14]:
def gradiente_descendiente(imagenes, alpha):
    imagenes_normal = imagenes[0]
    imagenes_pneumonia = imagenes[1]

    w = np.zeros(imagenes[0][0].shape) #elegir los valores, es un vector en R^k
    b = 0 #elegir los valores

    iteracion = 0

    while iteracion < MAX_ITER:
        print("Iteración: ", iteracion, "- Mínimo alcanzado hasta el momento: ", L(w, b))

        sum_dw = np.zeros(imagenes[0][0].shape)
        sum_db = 0

        for i in imagenes_normal:
            sum_dw += fAndG_w(b, 0, i, w)[1]
            sum_db += fAndG_b(b, 0, i, w)[1]
        
        for i in imagenes_pneumonia:
            sum_dw += fAndG_w(b, 1, i, w)[1]
            sum_db += fAndG_b(b, 1, i, w)[1]
        print("L viejo:", L(w, b))
        w_viejo = w
        b_viejo = b

        w = w - alpha * sum_dw
        b = b - alpha * sum_db
        print("L nuevo:", L(w, b))

        if(abs(L(w, b)-L(w_viejo, b_viejo)) < TOLERANCIA):
            break

        iteracion += 1

    return w, b

In [15]:
w_opt, b_opt = gradiente_descendiente(imagenes, 0.001)
print("El mínimo local ocurre en w = ", w_opt, "b = ", b_opt, " y L vale: ", L(w_opt, b_opt))

Iteración:  0 - Mínimo alcanzado hasta el momento:  674.5
L viejo: 674.5
L nuevo: 1342.0251637668782
Iteración:  1 - Mínimo alcanzado hasta el momento:  1342.0251637668782
L viejo: 1342.0251637668782
L nuevo: 829.1255040971772
Iteración:  2 - Mínimo alcanzado hasta el momento:  829.1255040971772
L viejo: 829.1255040971772
L nuevo: 1349.0
Iteración:  3 - Mínimo alcanzado hasta el momento:  1349.0
L viejo: 1349.0
L nuevo: 1349.0
El mínimo local ocurre en w =  [-0.06937209 -0.0971123  -0.1388036  ... -0.16313464 -0.03797094
  0.02773209] b =  -0.3758944776854393  y L vale:  1349.0


### Ejercicio 3

Utilizamos la funcion gradiente_descendiente para calcular el w optimo y b optimo de el conjunto de entrenamiento, este calcula los errores a medida que se van actualizando los valores de w y b

In [16]:
def error_cuadratico_medio(imagenes, w, b):
    imagenes_normal = imagenes[0]
    imagenes_pneumonia = imagenes[1]

    error_total = 0
    for i in imagenes_normal:
        error = (f(i, w, b) - 0)**2
        error_total += error
    
    for i in imagenes_pneumonia:
        error = (f(i, w, b) - 1)**2
        error_total += error

    return error_total/len(imagenes_normal + imagenes_pneumonia)

In [17]:
imagenes_normal = abrirImagenesEscaladas("chest_xray/train/NORMAL")
imagenes_pneumonia = abrirImagenesEscaladas("chest_xray/train/PNEUMONIA")
imagenes = [imagenes_normal, imagenes_pneumonia]

w_opt_train, b_opt_train, errores_train = gradiente_descendiente(imagenes, 0.001)
print(errores_train)

Iteración:  0 - Mínimo alcanzado hasta el momento:  1308.0
L viejo: 1308.0
L nuevo: 1349.0
Iteración:  1 - Mínimo alcanzado hasta el momento:  1349.0
L viejo: 1349.0
L nuevo: 1349.0


ValueError: not enough values to unpack (expected 3, got 2)

### Ejercicio 4

El parametro alpha es muy importante a la hora de encontrar el minimo. Este determina cuanto vamos a movernos en dirección al gradiente. Si este es muy chico, encontrar el minimo va a tardar muchos iteración y, dependiendo de el valor maximo, es posible que no se encuentre. Si se elige un valor muy grande, es posible que nos pasemos del valor minimo local.

In [ ]:
alpha = [0.0000001, 0.0001, 0.01, 1, 10]
for a in alpha:
    w_opt, b_opt = gradiente_descendiente(imagenes, a)
    print("Alpha: ", a, "- L = ", L(imagenes, w_opt, b_opt), " - Error cuadrático medio: ", error_cuadratico_medio(imagenes, w_opt, b_opt))

Iteración:  0 - Mínimo alcanzado hasta el momento:  60.67761335170323
w: [-3.06642722e-07 -4.30690102e-07 -4.30929655e-07 ...  1.15433243e-07
  1.55135632e-07  1.69193019e-07]
b: 0.4999963656900931
L nuevo: 53.07365823098891
L viejo: 60.67761335170323
diferencia L: 7.603955120714318
Iteración:  1 - Mínimo alcanzado hasta el momento:  53.07365823098891
w: [-1.81921134e-07 -4.30774881e-07 -4.34814820e-07 ...  4.02058690e-07
  5.08398319e-07  5.49468263e-07]
b: 0.4999940570316326
L nuevo: 50.42672157639585
L viejo: 53.07365823098891
diferencia L: 2.6469366545930626
Iteración:  2 - Mínimo alcanzado hasta el momento:  50.42672157639585
w: [ 3.04826898e-07 -5.81612696e-08 -6.84936028e-08 ...  8.09714086e-07
  1.00032642e-06  1.07714779e-06]
b: 0.4999930844972514
L nuevo: 49.771893872707835
L viejo: 50.42672157639585
diferencia L: 0.6548277036880137
Iteración:  3 - Mínimo alcanzado hasta el momento:  49.771893872707835
w: [9.30270381e-07 4.59395080e-07 4.41864898e-07 ... 1.25962754e-06
 1.540

KeyboardInterrupt: 

### Ejercicio 5

In [ ]:
escalado = [16, 32, 64, 128, 256, 512]

for e in escalado:
    imagenes_normal = abrirImagenesEscaladas("chest_xray/small_train/NORMAL", e)
    imagenes_pneumonia = abrirImagenesEscaladas("chest_xray/small_train/PNEUMONIA", e)
    imagenes = [imagenes_normal, imagenes_pneumonia]

    tiempo_inicial = time.time()
    w_opt, b_opt = gradiente_descendiente(imagenes, 0.001)
    tiempo_final = time.time()
    print("Tiempo de ejecución: ", tiempo_final-tiempo_inicial)
    print("Escala: ", e, "- L = ", L(imagenes, w_opt, b_opt), "- Error cuadrático medio: ", error_cuadratico_medio(imagenes, w_opt, b_opt))

Iteración:  0 - Mínimo alcanzado hasta el momento:  50.0
Iteración:  1 - Mínimo alcanzado hasta el momento:  47.37579165693172
Iteración:  2 - Mínimo alcanzado hasta el momento:  54.40794117059423
Iteración:  3 - Mínimo alcanzado hasta el momento:  96.60520972069014
Iteración:  4 - Mínimo alcanzado hasta el momento:  92.70278868357126
Iteración:  5 - Mínimo alcanzado hasta el momento:  69.75197786150365
Iteración:  6 - Mínimo alcanzado hasta el momento:  94.28938051439916
Iteración:  7 - Mínimo alcanzado hasta el momento:  79.86667296177323
Iteración:  8 - Mínimo alcanzado hasta el momento:  74.54716144111538
Iteración:  9 - Mínimo alcanzado hasta el momento:  87.4604925217846
Iteración:  10 - Mínimo alcanzado hasta el momento:  37.70919849237352
Iteración:  11 - Mínimo alcanzado hasta el momento:  37.5991346070351
Iteración:  12 - Mínimo alcanzado hasta el momento:  72.82041175355346
Iteración:  13 - Mínimo alcanzado hasta el momento:  87.60263641799033
Iteración:  14 - Mínimo alcanza

### Ejercicio 6